In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# This is my first published notebook and I think all concepts I used here is very simple for starter as I'm one too. Any comments will be appreciated.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.api.types import CategoricalDtype
sns.set(style='white', context='notebook', palette='deep', rc = {'figure.figsize':(15,8)})
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
train = train.drop("Id", axis=1)
y     = train["SalePrice"]
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
ID    = test["Id"]
test  = test.drop("Id",axis=1)
train_len  = len(train)
all_data = pd.concat((train,test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)


In [ ]:
print(train.shape)
train.head()

In [ ]:
types = train.dtypes
types[types != 'object']

In [ ]:
train.describe()

# Fill missing data


In [ ]:
mis_val = all_data.isnull().sum()
mis_val = mis_val[mis_val > 0]
mis_val.sort_values(ascending=False)

Alley, PoolQC, Fence, FireplaceQu, and MiscFeature is missing too much. However, from the data description, we can interpret that if those data is missing, it means that house does not have alley, pool, fence or other feature. Other missing features can be filled with the same assumption

In [ ]:
def fill_missing(df):
    df["Alley"]         = df["Alley"].fillna('None')
    df["Fence"]         = df["Fence"].fillna('None')
    df["PoolQC"]        = df["PoolQC"].fillna('None')
    df["MiscFeature"]   = df["MiscFeature"].fillna('None')
    df["FireplaceQu"]   = df["FireplaceQu"].fillna('None')
    df["LotFrontage"]   = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    df["GarageType"]    = df["GarageType"].fillna('None')
    df["GarageYrBlt"]   = df["GarageYrBlt"].fillna(0)
    df["GarageFinish"]  = df["GarageFinish"].fillna('None')
    df["GarageQual"]    = df["GarageQual"].fillna('None')
    df["GarageCond"]    = df["GarageCond"].fillna('None')
    df["BsmtQual"]      = df["BsmtQual"].fillna('None')
    df["BsmtCond"]      = df["BsmtCond"].fillna('None')
    df["BsmtExposure"]  = df["BsmtExposure"].fillna('None')
    df["BsmtFinType1"]  = df["BsmtFinType1"].fillna('None')
    df["BsmtFinType2"]  = df["BsmtFinType2"].fillna('None')
    df["MasVnrType"]    = df["MasVnrType"].fillna('None')
    df["MasVnrArea"]    = df["MasVnrArea"].fillna(0.0)
    df["Electrical"]    = df["Electrical"].fillna(df["Electrical"].mode()[0])
    df["Utilities"]     = df["Utilities"].fillna(df["Utilities"].mode()[0])
    df["BsmtFullBath"]  = df["BsmtFullBath"].fillna(df["BsmtFullBath"].mode()[0])
    df["BsmtHalfBath"]  = df["BsmtHalfBath"].fillna(df["BsmtHalfBath"].mode()[0])
    df["Functional"]    = df["Functional"].fillna(df["Functional"].mode()[0])
    df["Exterior1st"]   = df["Exterior1st"].fillna(df["Exterior1st"].mode()[0])
    df["Exterior2nd"]   = df["Exterior2nd"].fillna(df["Exterior2nd"].mode()[0])
    df["BsmtFinSF1"]    = df["BsmtFinSF1"].fillna(int(df["BsmtFinSF1"].mean()))
    df["BsmtFinSF2"]    = df["BsmtFinSF2"].fillna(int(df["BsmtFinSF2"].mean()))
    df["BsmtUnfSF"]     = df["BsmtUnfSF"].fillna(int(df["BsmtUnfSF"].mean()))
    df["TotalBsmtSF"]   = df["TotalBsmtSF"].fillna(int(df["TotalBsmtSF"].mean()))
    df["KitchenQual"]   = df["KitchenQual"].fillna(df["KitchenQual"].mode()[0])
    df["GarageCars"]    = df["GarageCars"].fillna(df["GarageCars"].mode()[0])
    df["GarageArea"]    = df["GarageArea"].fillna(int(df["GarageArea"].mean()))
    df["SaleType"]      = df["SaleType"].fillna(df["SaleType"].mode()[0])
    df["MSZoning"]      = df["MSZoning"].fillna(df["MSZoning"].mode()[0])
    
fill_missing(all_data)

Let check again if there is any missing data left

In [ ]:
mis_val = all_data.isnull().sum()
mis_val = mis_val[mis_val > 0]
mis_val.sort_values(ascending=False)

# LET DO SOME EDA


In [ ]:
g = sns.heatmap(train.corr())

I choose to plot this feature as it is the most correlated feature with sale price
There are 2 points look like outliers


In [ ]:
g = sns.scatterplot(train["GrLivArea"], train["SalePrice"])

In [ ]:
sns.distplot(y)


Sale price is skewed so we will take it log value to make it closer to norma

In [ ]:
y = np.log1p(y)

In [ ]:
sns.distplot(y);


Drop outliers 

In [ ]:
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

# Feature Engineering

MSubClass is actually categorical features so we will transform it

In [ ]:
#MSSubClass=The building class
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)

Let find features which have skewed distribution. Normal distribution has skew of 0 so I think transforming features with skewness > 0.6 is good enough and I have tried transforming features with skew > 0.3, which does not perform better.

In [ ]:
from scipy.stats import skew #for some statistics
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewed_feats = abs(skewed_feats)
skewed_feats = skewed_feats[skewed_feats > 0.6]
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.index

Ordered vars: Utilities, ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, HeatingQC, KitchenQual, Functional, FireplaceQu, GarageFinish, GarageQual, GarageCond, PoolQC, Fence. Ordered vars have important value about their order and should not be transformed into dummy vars.

In [ ]:
all_data["Utilities"]    = all_data["Utilities"].astype(CategoricalDtype(categories=['ELO','NoSeWa','NoSewr','AllPub'],ordered=True)).cat.codes
all_data["ExterQual"]    = all_data["ExterQual"].astype(CategoricalDtype(categories=['Po','Fa','TA','Gd','Ex'],ordered=True)).cat.codes
all_data["ExterCond"]    = all_data["ExterCond"].astype(CategoricalDtype(categories=['Po','Fa','TA','Gd','Ex'],ordered=True)).cat.codes
all_data["BsmtQual"]     = all_data["BsmtQual"].astype(CategoricalDtype(categories=['None','Po','Fa','TA','Gd','Ex'],ordered=True)).cat.codes
all_data["BsmtCond"]     = all_data["BsmtCond"].astype(CategoricalDtype(categories=['None','Po','Fa','TA','Gd','Ex'],ordered=True)).cat.codes
all_data["BsmtExposure"] = all_data["BsmtExposure"].astype(CategoricalDtype(categories=['None','No','Mn','Av','Gd'],ordered=True)).cat.codes
all_data["BsmtFinType1"] = all_data["BsmtFinType1"].astype(CategoricalDtype(categories=['None','Unf','LwQ','Rec','BLQ','ALQ','GLQ'],ordered=True)).cat.codes
all_data["BsmtFinType2"] = all_data["BsmtFinType2"].astype(CategoricalDtype(categories=['None','Unf','LwQ','Rec','BLQ','ALQ','GLQ'],ordered=True)).cat.codes
all_data["HeatingQC"]    = all_data["HeatingQC"].astype(CategoricalDtype(categories=['None','Po','Fa','TA','Gd','Ex'],ordered=True)).cat.codes
all_data["KitchenQual"]  = all_data["KitchenQual"].astype(CategoricalDtype(categories=['None','Fa','TA','Gd','Ex'],ordered=True)).cat.codes
all_data["Functional"]   = all_data["Functional"].astype(CategoricalDtype(categories=['Sal','Sev','Maj2','Maj1','Mod','Min2','Min1','Typ'],ordered=True)).cat.codes
all_data["FireplaceQu"]  = all_data["FireplaceQu"].astype(CategoricalDtype(categories=['None','Po','Fa','TA','Gd','Ex'],ordered=True)).cat.codes
all_data["GarageFinish"] = all_data["GarageFinish"].astype(CategoricalDtype(categories=['None','Unf','RFn','Fin'],ordered=True)).cat.codes
all_data["GarageQual"]   = all_data["GarageQual"].astype(CategoricalDtype(categories=['None','Po','Fa','TA','Gd','Ex'],ordered=True)).cat.codes
all_data["GarageCond"]   = all_data["GarageCond"].astype(CategoricalDtype(categories=['None','Po','Fa','TA','Gd','Ex'],ordered=True)).cat.codes
all_data["PoolQC"]       = all_data["PoolQC"].astype(CategoricalDtype(categories=['None','Fa','TA','Gd','Ex'],ordered=True)).cat.codes
all_data["Fence"]        = all_data["Fence"].astype(CategoricalDtype(categories=['None','MnWw','GdWo','MnPrv','GdPrv'],ordered=True)).cat.codes

Other vars is categorical so we will transform them into dummies vars

In [ ]:
all_data = pd.get_dummies(all_data,columns=["MSSubClass","Alley","BldgType","CentralAir",
"Condition1","Condition2","Electrical","Exterior1st","Exterior2nd","Foundation",
"GarageType","Heating","HouseStyle","LandContour","LandSlope","LotConfig","LotShape",
"MSZoning","MasVnrType","MiscFeature","Neighborhood","RoofMatl","RoofStyle",
"SaleCondition","SaleType","Street","MSSubClass",'MoSold','YrSold',"PavedDrive"],drop_first=True)
print(all_data.shape)

In [ ]:
skewed_features = ['MiscVal', 'PoolArea', 'LotArea', 'LowQualFinSF', '3SsnPorch',
       'KitchenAbvGr', 'BsmtFinSF2', 'EnclosedPorch', 'ScreenPorch',
       'BsmtHalfBath', 'MasVnrArea', 'OpenPorchSF', 'WoodDeckSF',
       'LotFrontage', '1stFlrSF', 'BsmtFinSF1', 'GrLivArea', 'TotalBsmtSF',
       'BsmtUnfSF', '2ndFlrSF', 'TotRmsAbvGrd', 'Fireplaces', 'HalfBath',
       'BsmtFullBath', 'GarageYrBlt']
for feature in skewed_features:
    all_data[feature] = np.log1p(all_data[feature])

Scaling features is usually good for prediction


In [ ]:
from sklearn.preprocessing import RobustScaler
N = RobustScaler()
all_data = N.fit_transform(all_data)

# Your favorite part: creating models

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, learning_curve
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
train = all_data[:train_len]
test  = all_data[train_len:]
X   = train
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X, y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfold, n_jobs=4))
    return (rmse)

In [ ]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,LassoCV, Ridge, LassoLars,ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
models = []
models.append(SVR())
models.append(RandomForestRegressor(max_depth=8, max_features ='auto', n_estimators=500))
models.append(KNeighborsRegressor())
models.append(DecisionTreeRegressor())
models.append(GradientBoostingRegressor())
models.append(AdaBoostRegressor())
models.append(LassoCV())
models.append(Ridge())
models.append(LassoLars())
models.append(ElasticNet())
cv_results = []
for model in models:
    cv_results.append(cv_rmse(model, X, y))
cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVR", "Random Forest", "KNN", 
"Decision Tree", "Gradient Boost", "Ada boost", "Lasso", "Ridge", "LassoLars","ElasticNet"]})
cv_res


In [ ]:
RFR = RandomForestRegressor(max_depth=8, max_features ='auto', n_estimators=500)
RFR.fit(X,y)    
# print(RFR.score(X_val, y_val))
# param_grid = { 
#     'n_estimators': [200, 500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,5,6,7,8]
# }

# CV_rfc = GridSearchCV(estimator=RFR, param_grid=param_grid, cv= 5,verbose=3)
# CV_rfc.fit(X, y)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
GBR = GradientBoostingRegressor(learning_rate=0.11058510725696458, max_leaf_nodes=50, n_estimators=200)
GBR.fit(X, y)
# param_grid = {
#     "n_estimators": [1, 2, 5, 10, 20, 50, 100, 200, 500],
#     "max_leaf_nodes": [2, 5, 10, 20, 50, 100],
#     "learning_rate": loguniform(0.01, 1),
# }
# search_cv = RandomizedSearchCV(
#     GradientBoostingRegressor(), param_distributions=param_grid,
#     scoring="neg_mean_absolute_error", n_iter=20, random_state=0, n_jobs=2, verbose=3
# )
# search_cv.fit(X_train, y_train)
# print(search_cv.best_score_)
# print(search_cv.best_params_)


In [ ]:
ridge = Ridge(alpha=1e-6) 
ridge.fit(X, y)

As we take log of the sale price so when you want to submit the prediction, you have to inverse it or you will have >9 score as I once had.

In [ ]:
y_pred = (GBR.predict(test) + ridge.predict(test) + RFR.predict(test))/3
sub = pd.DataFrame()
sub['Id'] = ID
sub['SalePrice'] = np.expm1(y_pred)
sub.to_csv('submission.csv',index=False)